# Mini-Project I
During this project, we will practice handling of complex lists and dictionaries in Python. Plus, we will learn how to work with API documentation. Don't be afraid to search for the information in the [**documentation**](https://api.tfl.gov.uk/swagger/ui/index.html?url=/swagger/docs/v1#!/AccidentStats/AccidentStats_Get).

Let's go to the tasks, we have some parsing to do :)!!

In [18]:
# import packages we need (remember what packages we used yesterday during the API session)
# IMPORT HERE
import requests
import os
import json

## Demo

In [19]:
# don't forget package os
app_id = os.environ['TFL_ID']
app_key = os.environ['TFL_KEY']
url_append = f'?app_id={app_id}&app_key={app_key}'

In [20]:
print(url_append)

?app_id=2c406994b5c9412286a6492231739a8b&app_key=4619fac400fb46a5a43531829dfbd029


In [21]:
# URL
url = "https://api.tfl.gov.uk/AirQuality"

In [22]:
# We send the request to the API
# NOTE: if you don't have your APP_KEY, run this without the url_append
res = requests.get(url+url_append)

In [23]:
# We can check if the request was successful
res.status_code

200

In [24]:
# We can export the information that was returned using method .json()
# res.json()

## Task
Parse the dictionary and print the AirQuality predictions for tomorrow

In [64]:
js_res = res.json()
# with open('air_res.json', 'w') as f:
#     f.write(json.dumps(js_res, indent=4))
# f.close()
print(js_res['currentForecast'][1]['forecastSummary'])

Low air pollution forecast valid from Saturday 12 March to end of Saturday 12 March GMT


## Task
What are the different modes of transport which are operated by Transfer for London? How many of modes do they have?

Print the list with different modes of transport, plus their count. Example output:
```
[bus, cable-car,.....]
Number of different modes of transport is: xyz
```

We need to search the documentation for correct request.

In [26]:
mode_url = 'https://api.tfl.gov.uk/Journey/Meta/Modes'
mode_res = requests.get(mode_url+url_append)
js_mode_res = mode_res.json()

# Operated by TFL

In [63]:
# with open('mode_res.json', 'w') as f:
#     f.write(json.dumps(js_mode_res, indent=4))
# f.close() <-- no need to close it
mode_list = []
n_modes = 0
for each_mode in js_mode_res:
    if each_mode['isTflService']:
        mode_list.append(each_mode['modeName'])
        n_modes += 1
print('[', end='')
for i in sorted(mode_list): print(f'{i}, ', end='')
print('\b\b]')
print(f'Number of different modes of transport is: {n_modes}')

[bus, cable-car, cycle-hire, dlr, elizabeth-line, overground, replacement-bus, river-bus, river-tour, tflrail, tram, tube]
Number of different modes of transport is: 12


## Task
How many BikePoints in London are operated by Transfor for London? How many docks are in **all** BikePoints? There is the information for empty and full docks for each BikePoint.

In [28]:
bike_point_url = 'https://api.tfl.gov.uk/BikePoint'
bike_point_res = requests.get(bike_point_url+url_append)
js_bike_point_res = bike_point_res.json()

## Check bike docks index
### [8] is not alway correct
### *safeguard makes things safer

In [69]:
# with open('bikepoint_res.json', 'w') as f:
#     f.write(json.dumps(js_bike_point_res, indent=4))
# f.close()

n_bike_point = 0
n_bike_docks = 0
n_empty_docks = 0
n_full_docks = 0
n_broken_docks = 0

for each_bp in js_bike_point_res:
    if each_bp['placeType'] == 'BikePoint':
        n_bike_point += 1

        for prop in each_bp['additionalProperties']:
            if prop['key'] == 'Nbdocks': bike_docks = int(prop['value'])
            elif prop['key'] == 'NbEmptyDocks': empty_bike_docks = int(prop['value'])
            elif prop['key'] == 'NbBikes': n_bike_at_dock = int(prop['value'])

        if bike_docks - (empty_bike_docks + n_bike_at_dock) == 0:
            n_full_docks += 1
        else:
            n_broken_docks += 1
        
        n_bike_docks += bike_docks
        n_empty_docks += empty_bike_docks

print(f'Total BikePoints = {n_bike_point}')
print(f'Total docks in all BikePoints = {n_bike_docks}')
print(f'There are {n_full_docks} BikePoints with full docks')
print(f'There are {n_empty_docks} empty docks available')
print(f'{n_broken_docks} docks are broken')

Total BikePoints = 788
Total docks in all BikePoints = 12608
There are 40 BikePoints with full docks
There are 9498 empty docks available
748 docks are broken


## Task
How many tube and bus lines are in London? Print names of all tube lines.

In [30]:
tube_bus_url = 'https://api.tfl.gov.uk/Line/Mode/bus,tube/Route'
tube_bus_res = requests.get(tube_bus_url+url_append)
js_tube_bus_res = tube_bus_res.json()

In [67]:
# with open('tubebus_res.json', 'w') as f:
#     f.write(json.dumps(js_tube_bus_res, indent=4))
# f.close()

n_bus_line = 0
n_tube_line = 0
tube_lines = []

for each_entry in js_tube_bus_res:
    if each_entry['modeName'] == 'bus': n_bus_line += 1
    if each_entry['modeName'] == 'tube':
        n_tube_line += 1
        tube_lines.append(each_entry['name'])

print(f'Total number of bus lines = {n_bus_line}, tube lines = {n_tube_line}')
print(f'Names of all tube lines:\n{tube_lines}')

Total number of bus lines = 678, tube lines = 11
Names of all tube lines:
['Bakerloo', 'Central', 'Circle', 'District', 'Hammersmith & City', 'Jubilee', 'Metropolitan', 'Northern', 'Piccadilly', 'Victoria', 'Waterloo & City']


## Task
How many station has `victoria` line?

In [32]:
victoria_in_url = 'https://api.tfl.gov.uk/Line/victoria/Route/Sequence/inbound'
victoria_out_url = 'https://api.tfl.gov.uk/Line/victoria/Route/Sequence/outbound'
victoria_in_res = requests.get(victoria_in_url+url_append)
victoria_out_res = requests.get(victoria_out_url+url_append)

In [33]:
n_victoria_in = len(victoria_in_res.json()['stations'])
n_victoria_out = len(victoria_out_res.json()['stations'])
print(f'Number of Victoria line inbound stations: {n_victoria_in}')
print(f'Number of Victoria line outbound stations: {n_victoria_out}')

Number of Victoria line inbound stations: 16
Number of Victoria line outbound stations: 16


## Task
Plan the journey from Heathrow Airport to Tower Bridge using Bus and Tube? Which way is faster? Example output:
```
Planned duration:
Bus: x minutes
Tube: y minutes
```

We need to search the documentation for correct requests and parameters we need.

## You can RESPOND to the ambiguity responses received from servers
### use the id of stops to figure out Depart Arrival points

In [90]:
url_append2 = url_append.replace('?', '&')

# bus only
plan_bus_url = 'https://api.tfl.gov.uk/Journey/JourneyResults/HillingdonHeathrowAirportTerminal3/to/TowerBridgecityhall?mode=bus&date=20220314&time=0800'
# tube only
plan_tube_url = 'https://api.tfl.gov.uk/Journey/JourneyResults/HillingdonHeathrowAirportTerminal3/to/TowerBridgecityhall?mode=tube&date=20220314&time=0800'

js_bus_res = requests.get(plan_bus_url+url_append2).json()
js_tube_res = requests.get(plan_tube_url+url_append2).json()

with open('bus_res.json', 'w') as f:
    f.write(json.dumps(js_bus_res, indent=4))
f.close()
with open('tube_res.json', 'w') as f:
    f.write(json.dumps(js_tube_res, indent=4))
f.close()

In [91]:
# bus+walking
plan_bus_url = 'https://api.tfl.gov.uk/Journey/JourneyResults/HillingdonHeathrowAirportTerminal3/to/TowerBridgecityhall?mode=bus,walking&date=20220314&time=0800'
# tube+walking
plan_tube_url = 'https://api.tfl.gov.uk/Journey/JourneyResults/HillingdonHeathrowAirportTerminal3/to/TowerBridgecityhall?mode=tube,walking&date=20220314&time=0800'

js_bus_res1 = requests.get(plan_bus_url+url_append2).json()
js_tube_res1 = requests.get(plan_tube_url+url_append2).json()

with open('busw_res.json', 'w') as f:
    f.write(json.dumps(js_bus_res1, indent=4))
f.close()
with open('tubew_res.json', 'w') as f:
    f.write(json.dumps(js_tube_res1, indent=4))
f.close()

## TFL ignores walking, try add walking mode mixed in bus/tube options

In [92]:
bus_journey_time = []
for j in js_bus_res['journeys']:
    bus_journey_time.append(j['duration'])
tube_journey_time = []
for j in js_tube_res['journeys']:
    tube_journey_time.append(j['duration'])

busw_journey_time = []
for j in js_bus_res1['journeys']:
    busw_journey_time.append(j['duration'])
tubew_journey_time = []
for j in js_tube_res1['journeys']:
    tubew_journey_time.append(j['duration'])

print('Planned duration:')
print(f'Bus: {min(bus_journey_time)} minutes')
print(f'Tube: {min(tube_journey_time)} minutes')
print(f'Bus+Walking: {min(busw_journey_time)} minutes')
print(f'Tube+Walking: {min(tubew_journey_time)} minutes')

Planned duration:
Bus: 187 minutes
Tube: 85 minutes
Bus+Walking: 187 minutes
Tube+Walking: 85 minutes
